In [16]:
import pandas as pd
import os
import cv2
import shutil
from imutils import face_utils
import cv2, glob, random, math, numpy as np, dlib, itertools

cwd = os.getcwd()
df = pd.read_csv(cwd + "/data/legend.csv")
df.to_csv(cwd + "/data_csv/all_data.csv")

df.drop("user_id", axis=1, inplace=True)
df["emotion"].replace({"anger": "ANGER", "contempt": "CONTEMPT", "disgust": "DISGUST", "fear": "FEAR", \
                        "happiness": "HAPPINESS", "neutral": "NEUTRAL", "sadness": "SADNESS", "surprise": "SURPRISE"}, inplace=True)
df = df.drop(df[df.emotion == "CONTEMPT"].index)
df = df.drop(df[df.emotion == "FEAR"].index)

In [17]:
df.groupby('emotion').count()

,image
emotion,
ANGER,252
DISGUST,208
HAPPINESS,5696
NEUTRAL,6868
SADNESS,268
SURPRISE,368


In [18]:
def variance_of_laplacian(image):
	return cv2.Laplacian(image, cv2.CV_64F).var()

cwd = os.getcwd()

dir_img = cwd + '/cleaned_images'
if os.path.exists(dir_img):
    shutil.rmtree(dir_img)
os.makedirs(dir_img)

all_image = []
count = 0
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(cwd + "/predictor/shape_predictor_68_face_landmarks.dat")
for idx, row in df.iterrows():
    
    imagePath = cwd + "/images/" + row.image

    image = cv2.imread(imagePath)
    height, width, channels = image.shape

    # check size image
    if(width != 350 or height != 350):
        continue

    # change color
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # check blurry
    fm = variance_of_laplacian(gray)
    if fm < 5:
        continue
    
    # detect face
    face_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_frontalface_alt.xml')
    eye_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_eye_tree_eyeglasses.xml')
    smile_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_smile.xml')

    face = face_cascade.detectMultiScale(
        gray,
        scaleFactor = 1.1,
        minNeighbors = 4,
        minSize = (200, 200),
        flags = cv2.CASCADE_SCALE_IMAGE
    )
    
    for (x, y, w, h) in face:
        roi_gray = gray[y:y+h, x:x+w]

    smile = smile_cascade.detectMultiScale(
        roi_gray,
        scaleFactor = 1.16,
        minNeighbors = 35,
        minSize = (25, 25),
        flags = cv2.CASCADE_SCALE_IMAGE
    )

    eyes = eye_cascade.detectMultiScale(roi_gray)

    

    if len(face) != 1 or len(smile) < 1 or len(eyes) < 2:
        continue
    
    rects = detector(image, 0)

    if len(rects) == 0:
        continue

    all_image.append([row.image, row.emotion])
    cv2.imwrite(os.path.join(dir_img, row.image), gray)
    count += 1

RuntimeError: Unable to open /Users/pongsakorn/Desktop/Workspace/python/facial_cleandata/predictor/shape_predictor_68_face_landmarks.dat

In [ ]:
new_df = pd.DataFrame(all_image, columns=["image", "emotion"])
new_df.to_csv(cwd + "/data_csv/preprocessing_data.csv")

In [ ]:
new_df.groupby('emotion').count()

,image
emotion,
ANGER,79
DISGUST,6
HAPPINESS,2379
NEUTRAL,2249
SADNESS,37
SURPRISE,87
